<a href="https://colab.research.google.com/github/IshanKashyap01/ML-training/blob/main/Doge_Coin_prediction_using_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recurrent Neural Network

## Part 1 - Data Preprocessing

### Importing the libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Importing the training set

In [15]:
dataset_train = pd.read_csv('/content/Dogecoin_train.csv')
training_set = dataset_train.iloc[:, 1:5].values
training_set

array([[2.46045470e-01, 2.66891350e-01, 2.40894350e-01, 2.64449680e-01],
       [2.37673280e-01, 2.55126670e-01, 2.30972110e-01, 2.44783900e-01],
       [2.63480250e-01, 2.89145320e-01, 2.30005650e-01, 2.39152930e-01],
       ...,
       [2.06889003e-04, 2.88790005e-04, 1.15593000e-04, 2.68539996e-04],
       [2.99312000e-04, 8.65873997e-04, 1.50213004e-04, 2.04730997e-04],
       [5.58776024e-04, 9.03883018e-04, 2.89601012e-04, 2.97880993e-04]])

### Feature Scaling

In [16]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

### Creating a data structure with 60 timesteps and 1 output

In [17]:
X_train = []
y_train = []
for i in range(60, 1258):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

### Reshaping

In [18]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

## Part 2 - Building and Training the RNN

### Importing the Keras libraries and packages

In [19]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

### Initialising the RNN

In [20]:
regressor = Sequential()

### Adding the first LSTM layer and some Dropout regularisation

In [21]:
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

### Adding a second LSTM layer and some Dropout regularisation

In [22]:
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

### Adding a third LSTM layer and some Dropout regularisation

In [23]:
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

### Adding a fourth LSTM layer and some Dropout regularisation

In [24]:
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

### Adding the output layer

In [25]:
regressor.add(Dense(units = 1))

### Compiling the RNN

In [26]:
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

### Fitting the RNN to the Training set

In [27]:
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100
38/38 [==============================] - 14s 151ms/step - loss: 8.6199e-04
Epoch 2/100
38/38 [==============================] - 5s 126ms/step - loss: 6.5540e-04
Epoch 3/100
38/38 [==============================] - 5s 124ms/step - loss: 6.6193e-04
Epoch 4/100
38/38 [==============================] - 5s 126ms/step - loss: 4.9674e-04
Epoch 5/100
38/38 [==============================] - 8s 220ms/step - loss: 6.4034e-04
Epoch 6/100
38/38 [==============================] - 11s 284ms/step - loss: 6.1898e-04
Epoch 7/100
38/38 [==============================] - 6s 157ms/step - loss: 6.6731e-04
Epoch 8/100
38/38 [==============================] - 5s 125ms/step - loss: 6.3778e-04
Epoch 9/100
38/38 [==============================] - 5s 124ms/step - loss: 4.1682e-04
Epoch 10/100
38/38 [==============================] - 5s 125ms/step - loss: 6.0323e-04
Epoch 11/100
38/38 [==============================] - 5s 126ms/step - loss: 4.2230e-04
Epoch 12/100
38/38 [==============================

## Part 3 - Making the predictions and visualising the results

### Getting the real stock price of 2017

In [28]:
dataset_test = pd.read_csv('/content/Doge_test.csv')
real_stock_price = dataset_test.iloc[:, 1:5].values

### Getting the predicted stock price of 2017

In [31]:
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, 80):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

ValueError: ignored

### Visualising the results

In [ ]:
plt.plot(real_stock_price, color = 'red', label = 'Real Google Stock Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Google Stock Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()